# Thinking
Thinking1	常见的规划问题都包括哪些？
        答：常见的规划问题包括：线性规划，例如在实际生产线上遇到的成本收益问题；整数线性规划，当决策变量都必须为整数时，线性规划问题则变成了整数线性规划，例如车辆运输问题，当车辆大小不一，且运输的货物只能按整箱来计算，这就涉及到整数线性规划问题了；混合整数规划是指部分决策变量为整数的线性规划问题，这种问题相比前面两种在实际应用中更加常见；车辆路径问题，指通过最少的时间、最短的路径来满足不同地点用户的不同需求的规划问题。
        
Thinking2	常用的规划工具包都有哪些？
        答：包括ortools、pulp、ortools等。
        
Thinking3	RFM模型的原理是怎样的？
        答：RFM模型是一种用户分层模型，它通过分析每一位用户的最近一次消费 (Recency)、消费频率 (Frequency)、消费金额 (Monetary)三个指标将用户划分到不同的策略等级上，包括：重要价值用户、重要发展用户、重要保持用户、重要挽留用户、一般价值用户、一般发展用户、一般保持用户、一般挽留用户这几类上，然后在用户管理上，根据用户的层级来进行不同的营销策略，从而使企业在相对较小的营销成本上获得更大的收益。


In [1]:
!pip install pulp -i https://pypi.tuna.tsinghua.edu.cn/simple

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [2]:
!pip install ortools -i https://pypi.tuna.tsinghua.edu.cn/simple

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [3]:
import numpy as np
from pprint import pprint
import pulp

In [6]:
def plant_problem(gain, x_max, y_max):
    row = len(gain)
    col = len(gain[0])
    print(f'row:{row} col:{col}')
    # 定义初始问题
    prob = pulp.LpProblem('plant', sense=pulp.LpMaximize)
    # 定义相关变量
    var = [[pulp.LpVariable(f'x{i}{j}', lowBound=0, cat=pulp.LpInteger) for j in range(col)] for i in range(row)]
    # 递归展开列表
    flatten = lambda x: [y for l in x for y in flatten(l)] if type(x) is list else [x]
    print('var=', var)
    print('flatten=', flatten)
    print('var[1]', var[1])
    # 定义目标函数 （单价* {ij}运量）
    prob += pulp.lpDot(flatten(var), gain.flatten())
    # 定义约束条件
    for i in range(row):
        prob += pulp.lpSum(var[i]) <= x_max[i]
    for j in range(col):
        prob += pulp.lpSum([var[i][j] for i in range(row)]) <= y_max[j]

    prob.solve()
    result = {'objective':pulp.value(prob.objective), \
              'var': [[pulp.value(var[i][j]) for j in range(col)] for i in range(row)]}
    return result


In [8]:
# 耕地播种策略问题
if __name__ == '__main__':
#不同土地上播种不同作物的产量
    gain = np.array([[500, 550, 630, 1000, 800, 700],
                      [800, 700, 600, 950, 900, 930],
                      [1000, 960, 840, 650, 600, 700],
                      [1200, 1040, 980, 860, 880, 780]])
    # 各种作物的计划播种面积
    max_plant = [76, 88, 96, 40]
    # 各个地块的面积
    max_cultivation = [42, 56, 44, 39, 60, 59]
    # 使用线性规划
    result = plant_problem(gain, max_plant, max_cultivation)
# 输出结果
    print(f'最大值为{result["objective"]}')
    print('各变量的取值为：')
    pprint(result['var'])
#从结果可以看出要想获得最好的收益，方案为：
#1.小麦在3号地播种6亩，在4号地39亩，在5号地31亩
#2.玉米在5号播种29，6号播种59
#3.水果在1号播种2亩，2号播种56亩，3号播种38亩
#4.蔬菜在1号播种40亩

row:4 col:6
var= [[x00, x01, x02, x03, x04, x05], [x10, x11, x12, x13, x14, x15], [x20, x21, x22, x23, x24, x25], [x30, x31, x32, x33, x34, x35]]
flatten= <function plant_problem.<locals>.<lambda> at 0x7f9c68492680>
var[1] [x10, x11, x12, x13, x14, x15]
最大值为284230.0
各变量的取值为：
[[0.0, 0.0, 6.0, 39.0, 31.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 29.0, 59.0],
 [2.0, 56.0, 38.0, 0.0, 0.0, 0.0],
 [40.0, 0.0, 0.0, 0.0, 0.0, 0.0]]
